In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('trees').getOrCreate()

24/11/02 20:08:31 WARN Utils: Your hostname, AlienEE resolves to a loopback address: 127.0.1.1; using 10.0.0.177 instead (on interface wlp4s0)
24/11/02 20:08:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/02 20:08:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.ml import Pipeline

In [5]:
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier,
                                       DecisionTreeClassifier)

In [ ]:
data = spark.read.format('libsvm').load('datasets/sample_libsvm_data.txt')

24/11/02 20:11:25 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [8]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [9]:
data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                100|
|   mean|               0.57|
| stddev|0.49756985195624287|
|    min|                0.0|
|    max|                1.0|
+-------+-------------------+



In [10]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [11]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [12]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [13]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [14]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[100,101,102...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|   [30.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[154,155,156...|   [0.0,43.0]|  [0.0,1.0]|       1.0|
|  0.0|(69

In [15]:
gbt_preds.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[123,124,125...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[123,124,125...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[123,124,125...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[124,125,126...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[124,125,126...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[124,125,126...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[126

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [20]:
print(f"DTC Accuracy {acc_eval.evaluate(dtc_preds)}")


DTC Accuracy 0.9259259259259259


In [21]:
print(f"RFC Accuracy {acc_eval.evaluate(rfc_preds)}")


RFC Accuracy 1.0


In [22]:
print(f"GBT Accuracy {acc_eval.evaluate(gbt_preds)}")


GBT Accuracy 0.9259259259259259


In [24]:
rfc_model.featureImportances

SparseVector(692, {99: 0.0002, 100: 0.0005, 124: 0.0015, 126: 0.0003, 178: 0.0004, 184: 0.0014, 187: 0.0004, 208: 0.0003, 209: 0.001, 212: 0.0005, 216: 0.0094, 240: 0.0006, 243: 0.0011, 261: 0.0045, 271: 0.0011, 272: 0.008, 274: 0.0039, 288: 0.0006, 289: 0.0088, 290: 0.0079, 300: 0.0248, 301: 0.0006, 315: 0.0006, 316: 0.0017, 318: 0.006, 319: 0.0002, 322: 0.001, 323: 0.0074, 324: 0.0006, 327: 0.001, 328: 0.0002, 329: 0.0013, 343: 0.0058, 347: 0.0011, 350: 0.0103, 351: 0.0261, 353: 0.0006, 354: 0.0038, 356: 0.0091, 357: 0.0107, 358: 0.0119, 359: 0.0038, 360: 0.0006, 371: 0.0073, 373: 0.0004, 377: 0.0162, 378: 0.0117, 379: 0.0136, 380: 0.0029, 384: 0.0212, 386: 0.0061, 388: 0.0005, 399: 0.0072, 400: 0.0209, 401: 0.0081, 405: 0.008, 406: 0.0144, 407: 0.0511, 408: 0.0006, 412: 0.003, 414: 0.006, 425: 0.0042, 426: 0.0034, 427: 0.0073, 428: 0.0005, 429: 0.0086, 433: 0.0264, 434: 0.0278, 435: 0.0082, 439: 0.0017, 440: 0.0008, 441: 0.0217, 443: 0.0024, 454: 0.0055, 455: 0.0128, 456: 0.0208, 45